In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

from pathlib import Path
from PIL import Image, ImageDraw, ImageOps
from IPython.display import display

In [3]:
_RAW_TRAIN_IMAGE_DIRECTORY = Path("../data/images/train-raw/")
_RAW_TRAIN_LABEL_DIRECTORY = Path("../data/labels/train-raw/")

_TRAIN_IMAGE_DIRECTORY = Path("../data/images/train/")
_TRAIN_LABEL_DIRECTORY = Path("../data/labels/train/")

In [4]:
def create_image_list(directory):
    for root, dirs, files in os.walk(directory):
        return files

train_images = create_image_list(_RAW_TRAIN_IMAGE_DIRECTORY)
train_labels = create_image_list(_RAW_TRAIN_LABEL_DIRECTORY)
print("Amount of train images:", len(train_images))
print("Length of images is equal to labels:", len(train_images) == len(train_labels))

Amount of train images: 13386
Length of images is equal to labels: True


In [5]:
def parse_annotations_yolo(annotions_file_path, image_size):
    image_width, image_height = image_size
    annotations = []
    with open(annotions_file_path, 'r') as file:
        for box_annotation in file:
            coordinates = box_annotation.strip().split()[1:]
            x_center, y_center, width, height = map(float, coordinates)
            x_min = int((x_center - width / 2) * image_width)
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width)
            y_max = int((y_center + height / 2) * image_height)
            annotations.append(((x_min, y_min), (x_max, y_max)))
    return np.array(annotations, dtype=np.float64)

In [6]:
def display_image(image):
  fig = plt.figure(figsize=(5, 5))
  plt.grid(False)
  plt.imshow(image)

def show_boxes_on_image(image, annotations):
    draw = ImageDraw.Draw(image)
    for ((x_min, y_min), (x_max, y_max)) in annotations:
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
    image.show()

In [7]:
def resize_image(image, annotations, new_size = (300, 300)):
    image, annotations = expand_to_square(image, annotations, new_size)
    new_image = ImageOps.fit(image, new_size, Image.LANCZOS)
    return new_image, annotations


def expand_to_square(image, annotations, new_size, background_color=(255, 255, 255)):
    """
    Expands rectangular image to square by adding a white border
    """
    width, height = image.size
    if width == height:
        return image
    elif width > height:
        result = Image.new(image.mode, (width, width), background_color)
        result.paste(image, (0, (width - height) // 2))
        new_width, new_height = result.size
        annotations = expand_annotations(annotations, width, height, new_width, new_height, new_size)
        return result, annotations
    else:
        result = Image.new(image.mode, (height, height), background_color)
        result.paste(image, ((height - width) // 2, 0))
        new_width, new_height = result.size
        annotations = expand_annotations(annotations, width, height, new_width, new_height, new_size)
        return result, annotations

def expand_annotations(annotations, old_width, old_height, new_width, new_height, new_size):
    if x_axis_change := new_width - old_width:
        for annotation in annotations: 
            annotation[:, 0] += x_axis_change / 2
    if y_axis_change := new_height - old_height:
        for annotation in annotations: 
            annotation[:, 1] += y_axis_change / 2 
    annotation[:, 0] *= new_size[0] / new_width
    annotation[:, 1] *= new_size[1] / new_height
    return annotations
        

In [8]:
input_shape = (300, 300, 1)
image_nr = 6

In [9]:
image_path = _RAW_TRAIN_IMAGE_DIRECTORY/train_images[image_nr]
annotations_file_path = _RAW_TRAIN_LABEL_DIRECTORY/train_labels[image_nr]
image = Image.open(image_path)

annotations = parse_annotations_yolo(annotations_file_path, image.size) 
image, annotations = resize_image(image, annotations, (input_shape[0], input_shape[1]))
show_boxes_on_image(image, annotations)

In [10]:
x_train = []
y_train = []

# for image_index in range(len(train_images)):
for image_index in range(100):
    image_path = _RAW_TRAIN_IMAGE_DIRECTORY/train_images[image_nr]
    annotations_file_path = _RAW_TRAIN_LABEL_DIRECTORY/train_labels[image_nr]

    image = Image.open(image_path)
    annotations = parse_annotations_yolo(annotations_file_path, image.size) 
    image, annotations = resize_image(image, annotations, (input_shape[0], input_shape[1]))

    image_path_processed = _TRAIN_IMAGE_DIRECTORY/train_images[image_index]
    # label_path_processed = _TRAIN_LABEL_DIRECTORY/train_labels[image_index]
    
    image.save(image_path_processed)

    x_train.append(image_path_processed)
    y_train.append(annotations)

In [ ]:
model = keras.Sequential([
    keras.Input(shape=input_shape),
    keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu'),
    keras.layers.AveragePooling2D(2,2),

    keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    keras.layers.AveragePooling2D(2,2),

    keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    keras.Dropout(0.2),
    keras.layers.AveragePooling2D(2,2),
    keras.layers.Dense(units = 4),
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.FalseNegatives(),
    ],
)

